In [ ]:
# dataset
# https://www.kaggle.com/datasets/mathurinache/math-dataset
# !kaggle datasets download -d mathurinache/math-dataset

In [1]:
!kaggle datasets download -d mathurinache/math-dataset
!unzip math-dataset.zip
!pip install datasets
!pip install transformers
!pip install peft
!pip install trl

from IPython import display
display.clear_output()

In [3]:
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, prepare_model_for_kbit_training, LoraConfig
from trl import SFTTrainer

In [4]:
LIMIT = 1000
CONTEXTWINDOW = 512

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
# Load model directly

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
EOS_TOKEN = tokenizer.eos_token
tokenizer.pad_token = EOS_TOKEN
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
def tokenize_function(examples):

   return tokenizer(examples['text'],
                    padding="max_length",
                    max_length=CONTEXTWINDOW,
                    truncation=True,
                    return_tensors='pt')

In [7]:
alpaca_prompt = """Below is a Maths Question. Write an answer that appropriately completes the request.

### Question:
{}

### Answer:
"""

def generate_prompt(examples):
    problems  = examples["problem"]
    solutions = examples["solution"]
    texts = []
    for problem, solution in zip(problems, solutions):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!

        texts.append(
            alpaca_prompt.format(problem) + solution + EOS_TOKEN
        )
    return { "text" : texts, }

In [18]:
dataset = load_dataset("json", data_dir='./MATH')
dataset['train'] = dataset['train'].map(generate_prompt, batched=True)
# tokenized_dataset = dataset.map(tokenize_function, batched=True).with_format("torch")

Resolving data files:   0%|          | 0/7500 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/5000 [00:00<?, ?it/s]

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

In [ ]:
""" Sample """

# input_tokens   = tokenized_dataset["test"][0]["input_ids"].unsqueeze(0).to(DEVICE)
# attention_mask = tokenized_dataset["test"][0]["attention_mask"].unsqueeze(0).to(DEVICE)

# generation_output = model.generate(
#       input_ids=input_tokens,
#       max_new_tokens=1000,
#       attention_mask=attention_mask,
#       temperature=0.3,
#       eos_token_id=EOS_TOKEN,
#     )

In [ ]:
# dataset["test"][0]['text']

In [ ]:
# tokenizer.decode(generation_output[0])

In [49]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [9]:
lora_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules=["q_proj","k_proj","v_proj","o_proj"],
        bias="none",
        task_type="CAUSAL_LM",
    )

In [10]:
# Loading in 8 bit ..."
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

In [11]:
output_dir = "cp"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
per_device_eval_batch_size = 4
eval_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 10
learning_rate = 5e-4
max_grad_norm = 0.3
max_steps = 5
warmup_ratio = 0.03
evaluation_strategy="steps"
lr_scheduler_type = "constant"

training_args = transformers.TrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            optim=optim,
            eval_strategy=evaluation_strategy,
            save_steps=save_steps,
            learning_rate=learning_rate,
            logging_steps=logging_steps,
            max_grad_norm=max_grad_norm,
            max_steps=max_steps,
            warmup_ratio=warmup_ratio,
            group_by_length=True,
            lr_scheduler_type=lr_scheduler_type,
            ddp_find_unused_parameters=False,
            eval_accumulation_steps=eval_accumulation_steps,
            per_device_eval_batch_size=per_device_eval_batch_size,
        )

In [20]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    dataset_text_field='text',
    peft_config=lora_config,
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_args
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [22]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 3.6 MB/s eta 0:00:00


In [ ]:
trainer.train()
trainer.save_model(f"{output_dir}/final")